In [45]:
import sys
import pandas as pd
from tkinter import *
from tkinter import messagebox
import tkinter.ttk as ttk   
import datetime
import csv
import requests
import json
import datetime

In [46]:
class Main(Tk):
    def __init__(self):
        super().__init__()
        self.title('NotebookSample')
        self.geometry('500x500')
        self.create_widgets()
        self.mainloop()
    def create_widgets(self):
        self.note = Note1(self)
        self.note.place(x=100, y=0, width=400, height=500)

In [47]:
class Note1(ttk.Notebook):
    def __init__(self,master=None):
        super().__init__(master)
        self.create_widgets()
    def create_widgets(self):
        #tab1
        frame1 = Frame1(self)
        self.add(frame1, text='今日の出来事')
        #tab2
        frame2 = Frame2(self)
        self.add(frame2, text='日記の検索')
        #tab3
        frame3 = Frame3(self)
        self.add(frame3, text='今日の天気')

In [48]:
#日記
class Frame1(ttk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.create_widgets()
    def create_widgets(self):
        title_label = Label(self, text='日付')
        self.entry = Entry(self)
        today = get_today()
        self.entry.insert(0,today)
        self.text = Text(self, width=45)
        btn_1 = Button(self, text= '送信')
        btn_1.bind('<Button-1>',self.button_tapped)

        title_label.place(x=0, y=0)
        self.entry.place(x=50, y=0)
        self.text.place(x=0, y=40)
        btn_1.place(x=0, y=400)
    def button_tapped(self,button):
        self.display_message()
        date = self.get_date()
        text = self.get_text()
        save_file(date, text)
    def display_message(self):
        messagebox.showinfo('message','日記を記録しました')
    def get_date(self):
        return self.entry.get()
    def get_text(self):
        return self.text.get('1.0','end-1c')

def save_file(date, text):
    if os.path.exists('./diary.csv'):
        with open('./diary.csv', 'a', newline='') as f:
            w = csv.writer(f, delimiter=',')
            w.writerow([date, text])
    else: 
        ans = messagebox.askyesno('確認','フォルダが存在しません。作成しますか？')
        if ans == True:
            with open('./diary.csv', 'a',newline='') as f:
                w = csv.writer(f, delimiter=',')
                w.writerow([date, text])
        else:
            print('Fault')

def get_today():
    today = datetime.datetime.now()
    return today.strftime('%Y/%m/%d')



In [49]:
#日記検索
class Frame2(ttk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.set_data()
        self.create_widgets()

    def create_widgets(self):
        label = Label(self, text='キーワード')
        self.entry = Entry(self)
        # text = Text(self, width=45)
        button = Button(self, text='検索')
        button.bind('<Button-1>',self.button_tapped)

        label.place(x=0,y=0)
        self.entry.place(x=100, y=0)
        # text.place(x=0, y=40)
        button.place(x=0, y=400)

        tree = self.create_tree()
        tree.place(x=0, y=40)

    def button_tapped(self, event=None):
        self.search_csv()

    def get_entry(self):
        return self.entry.get()

    def set_data(self):
        self.df = pd.read_csv('./diary.csv')
        self.colname_list = ['日付','出来事']
        self.width_list = [100,250]
        self.search_col = '出来事'

    def search_csv(self):
        keyword = self.get_entry()
        result = self.df[self.df[self.search_col].str.contains(keyword, na=False)]
        self.update_tree_by_search_result(result)

    def update_tree_by_search_result(self, result):
        self.tree.delete()
        for _, row in result.iterrows():
            self.tree.insert("", "end", values=row[self.colname_list].to_list())
            
    def create_tree(self):
        self.tree = ttk.Treeview(self, height=15)
        self.tree["columns"] = self.colname_list
        self.tree["show"] = "headings"

        for i, (colname, width) in enumerate(zip(self.colname_list, self.width_list)):
            self.tree.heading(i, text=colname)
            self.tree.column(i, width= width)

        return self.tree



In [50]:
#今日の天気
class Frame3(ttk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.crete_widgets()
    def crete_widgets(self):
        self.label = Label(self, text='都市名を入れてください。例：名古屋市')
        self.entry_city = Entry(self)
        self.result_button = Button(self, text='result')
        self.reset_button = Button(self, text='reset')

        self.result_button.bind('<Button-1>', self.result_button_tapped)
        self.reset_button.bind('<Button-1>', self.reset_button_tapped)

        self.label.place(x=0, y=0)
        self.entry_city.place(x=0, y=30)
        self.result_button.place(x=0,y=60)
        self.reset_button.place(x=100, y=60)
    
        tree = self.create_tree()
        tree.place(x=0, y=90)
    def result_button_tapped(self, button):
        city = self.entry_city.get()
        self.show_result(city)
    def reset_button_tapped(self, button):
        for i in self.tree.get_children():
            self.tree.delete(i)

    def create_tree(self):
        self.tree = ttk.Treeview(self, height=15)
        self.tree["columns"] = (1,2,3)
        self.tree["show"] = "headings"
        self.tree.heading(1, text='datetime')
        self.tree.heading(2, text='weather')
        self.tree.heading(3, text = 'temperature(℃)')
        self.tree.column(1, width=100)
        self.tree.column(2, width=100)
        self.tree.column(3, width=120)
        return self.tree
    
    def update_5days_screen(self, datas):
        for i in range(len(datas)):
            self.tree.insert("","end",values=(datas['datetime'][i], datas['weather'][i], datas['temperature'][i]))

    def show_result(self, city):
        try:
            # APIを受け取る
            json = get_json(city)
            # 必要データをAPIから取得
            datas = get_5days_dates(json)
            self.update_5days_screen(datas)
        except TypeError:
            self.label['text'] = '該当するものがありませんでした。「市」のつけ忘れなどはありませんか。'


In [51]:

def get_json(city):
    with open('./ignore.txt', 'r+') as f:
        info = f.readlines()
        url = info[0].replace('\n','')
        key = info[1]
    url = url.format(city=city, key=key)
    try:
        response = requests.get(url)
        return response.json()
    except:
        print('Not found.')

def get_dates(json):
    country = json["sys"]["country"]
    city = json["name"]
    weather = json["weather"][0]['main']
    temp = json["main"]["temp"]
    return [country, city, weather, temp]

#三時間ごとの表示をするため
def get_now_time(hour=3):
    now = datetime.datetime.today()
    return str(now.month) + ('/') + str(now.day) + (' ') + str(now.hour - (now.hour % 3)).zfill(2) + ":00"

def create_dataframe(datas):
    df = pd.DataFrame(datas,columns=['datetime','weather','temperature'])
    return df

def get_5days_dates(json):
    now = get_now_time(hour=3)
    datas = []
    target_index = 0
    try:
        for i, a_day in enumerate(json['list']):
            date = a_day['dt_txt'][6:-3]
            weather = a_day['weather'][0]['description']
            temperature = a_day['main']['temp']
            date = date.replace('-','/')
            datas.append([date, weather,temperature])
            if date == now:
                target_index = i
        return create_dataframe(datas[target_index:])
    except KeyError:
        print('該当がありませんでした')


In [52]:
main = Main()